Ex_1

In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

a) Prepare data

In [59]:
url_data = "https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data"
df = pd.read_csv(url_data, delimiter='\t')
df

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T
...,...,...,...,...,...,...,...,...,...,...,...
92,93,2.830268,3.876396,68,-1.386294,1,1.321756,7,60,4.385147,T
93,94,3.821004,3.896909,44,-1.386294,1,2.169054,7,40,4.684443,T
94,95,2.907447,3.396185,52,-1.386294,1,2.463853,7,10,5.143124,F
95,96,2.882564,3.773910,68,1.558145,1,1.558145,7,80,5.477509,T


b) Extract and normalize the explicative variables

In [60]:
variables = df.columns[1:9]
df[variables] = df[variables].apply(lambda x: (x - x.mean()) / x.std())

<!-- Is it wise to normalize these data? -->c) --> Yes it is because all the variables might be on the same scale

d) Split the dataset into training and test data

In [61]:
Y_train = df.loc[df["train"]=="T", 'lpsa'].to_numpy()
X_train = df.loc[df["train"]=="T", variables].to_numpy()
print("Training set : n = {} samples and p = {} dimensions".format(X_train.shape[0],X_train.shape[1]))
Y_test = df.loc[df["train"]=="F", 'lpsa'].to_numpy()
X_test = df.loc[df["train"]=="F", variables].to_numpy()
print("Test set : n = {} samples and p = {} dimensions".format(X_test.shape[0], X_test.shape[1]))

Training set : n = 67 samples and p = 8 dimensions
Test set : n = 30 samples and p = 8 dimensions


2. Recherchez la solution du lasso telle qu’elle a été publiée dans le livre (comme dans le TP1)

In [42]:
import cvxpy as cvx
Xn = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
yn = Y_train - Y_train.mean()
n, p = Xn.shape
lamda = 23.39
b = cvx.Variable(p)
obj = cvx.Minimize(cvx.sum_squares(Xn@b - yn) + lamda*cvx.norm(b, 1))
prob = cvx.Problem(obj)
prob.solve(solver=cvx.SCS, eps=1e-5)

b.value = np.round(b.value, 4)
print("solutio Lasso :", b.value)
print("coefficients non nuls :", np.abs(b.value) > 1e-5)

# calculer les prédictions sur le jeu de test
X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0)
y_pred = X_test @ b.value + Y_train.mean()
mse = np.mean((y_pred - Y_test)**2)
print("MSE sur le jeu de test :", mse)

solutio Lasso : [ 0.5619  0.2002 -0.      0.0348  0.1188 -0.     -0.      0.0201]
coefficients non nuls : [ True  True False  True  True False False  True]
MSE sur le jeu de test : 0.46328012580657446


3. Recherchez la solution de l’elastic net

In [43]:
# 3. Recherchez la solution de l’elastic net
# Jel(β) = 1/2 ‖Xβ − y‖2 + λ‖β‖1 + γ/2 ‖β‖2
# a) resolution avec cvxpy et lamda = 6 et gamma = 0.005

lamda = 6
gamma = 0.005
b = cvx.Variable(p)
obj = cvx.Minimize(1/2 * cvx.sum_squares(Xn@b - yn) + lamda * cvx.norm(b, 1) + gamma/2 * cvx.norm(b, 2)**2)
prob = cvx.Problem(obj)
prob.solve(solver=cvx.SCS, eps=1e-5)

b.value = np.round(b.value, 4)
print("solution Elastic Net :", b.value)
print("coefficients non nuls :", np.abs(b.value) > 1e-5)
#prédictions sur le jeu de test
X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0)
y_pred = X_test @ b.value + Y_train.mean()
mse = np.mean((y_pred - Y_test)**2)
print("MSE sur le jeu de test :", mse)

solution Elastic Net : [ 0.5718  0.2326  0.      0.1148  0.1783 -0.      0.      0.0717]
coefficients non nuls : [ True  True False  True  True False False  True]
MSE sur le jeu de test : 0.45901539555975385


In [44]:
# b)comme un problème de lasso
lamda = 6
b = cvx.Variable(p)
obj = cvx.Minimize(1/2 * cvx.sum_squares(Xn@b - yn) + lamda * cvx.norm(b, 1))
prob = cvx.Problem(obj)
prob.solve(solver=cvx.SCS, eps=1e-5)

b.value = np.round(b.value, 4)
print("solution Lasso :", b.value)
print("coefficients non nuls :", np.abs(b.value) > 1e-5)
#prédictions sur le jeu de test
X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0)
y_pred = X_test @ b.value + Y_train.mean()
mse = np.mean((y_pred - Y_test)**2)
print("MSE sur le jeu de test :", mse)
RMSE = np.sqrt(mse)
print("RMSE sur le jeu de test :", RMSE)

solution Lasso : [ 0.5719  0.2326  0.      0.1148  0.1783 -0.      0.      0.0716]
coefficients non nuls : [ True  True False  True  True False False  True]
MSE sur le jeu de test : 0.466035193114146
RMSE sur le jeu de test : 0.6826677032891961


In [62]:
Xn = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
yn = Y_train - Y_train.mean()

In [63]:
lamda = 3 

b = cvx.Variable(p)
lasso_obj = cvx.Minimize(0.5 * cvx.sum_squares(Xn @ b - yn) + lamda * cvx.norm(b, 1))
lasso_prob = cvx.Problem(lasso_obj)
lasso_prob.solve(solver=cvx.SCS, eps=1e-5)
beta_lasso = b.value
print("β̂_Lasso =", np.round(beta_lasso, 4))

β̂_Lasso = [ 0.5836  0.2558 -0.0333  0.1632  0.208  -0.0036  0.      0.1078]


In [65]:

lamda = 3

b = cvx.Variable(p)
lasso_obj = cvx.Minimize(0.5 * cvx.sum_squares(Xn @ b - yn) + lamda * cvx.norm(b, 1))
lasso_prob = cvx.Problem(lasso_obj)
lasso_prob.solve(solver=cvx.SCS, eps=1e-5)
beta_lasso = np.array(b.value).flatten()
print("β̂_Lasso :", np.round(beta_lasso, 4))


w = 1 / np.sqrt(np.abs(beta_lasso) )
W_inv = np.diag(1 / w)  


X_new = Xn / w[np.newaxis, :] 

alpha = cvx.Variable(p)
obj_adapt = cvx.Minimize(0.5 * cvx.sum_squares(X_new @ alpha - yn) + lamda * cvx.norm(alpha, 1))
prob_adapt = cvx.Problem(obj_adapt)
prob_adapt.solve(solver=cvx.SCS, eps=1e-5)
alpha_hat = np.array(alpha.value).flatten()
print("α̂ :", np.round(alpha_hat, 4))

beta_adapt = alpha_hat / w
print("β̂ adaptatif :", np.round(beta_adapt, 4))
print("Coefficients non nuls :", np.abs(beta_adapt) > 1e-5)

X_test_n = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0)
y_pred = X_test_n @ beta_adapt + Y_train.mean()
mse = np.mean((y_pred - Y_test)**2)
print("MSE sur le jeu de test :", mse)


β̂_Lasso : [ 0.5836  0.2558 -0.0333  0.1632  0.208  -0.0036  0.      0.1078]
α̂ : [0.8595 0.4612 0.     0.2022 0.3271 0.     0.     0.    ]
β̂ adaptatif : [0.6566 0.2333 0.     0.0817 0.1492 0.     0.     0.    ]
Coefficients non nuls : [ True  True False  True  True False False False]
MSE sur le jeu de test : 0.4429260605754737
